In [28]:
from typing import Tuple

import np

import GNSS
!!auto-py-to-exe

^C


[]

In [2]:
N = 3
import numpy as np
B1 = np.zeros((1,N))
B2 = np.zeros((N, 1))
B3 = np.zeros(N)
U = 0
U1 = np.zeros(1)


In [4]:
B2 @ U

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [35]:
# import pandas as pd
# gpa = pd.read_csv('sfrbx_gps_alm.csv', index_col=0)
# gpe = pd.read_csv('sfrbx_gps_eph.csv', index_col=0)
# gla = pd.read_csv('sfrbx_glonass_alm.csv', index_col=0)
# gle = pd.read_csv('sfrbx_glonass_eph.csv', index_col=0)
# with open('sfrbx_gps_data.txt', 'r') as file:
#     gp_data = eval(file.read())

In [34]:
# import pickle
# from datetime import datetime
# import Constants
# t1 = datetime.now(tz=Constants.tz_utc)
# SFRBX_GLONASS_PATH = 'glonass_sfrbx.plk'
# with open(SFRBX_GLONASS_PATH, 'wb') as file:
#     pickle.dump(gpe, file)
#     pickle.dump(gpa, file)
#     pickle.dump(gle, file)
#     pickle.dump(gla, file)
#     pickle.dump(gp_data, file)
# t2 = datetime.now(tz=Constants.tz_utc)
# print(t2-t1)  

0:00:00.002186


In [43]:
import pandas as pd

# SFRBX_GLONASS_PATH = 'glonass_sfrbx.plk'
SFRBX_path = 'sfrbx_data.plk'
import pickle
with open(SFRBX_path, 'rb') as file:
    gpe = pickle.load(file)
    gpa = pickle.load(file)
    gle = pickle.load(file)
    gla = pickle.load(file)
    gpD = pickle.load(file)
    glD = pickle.load(file)
# gla1 == gla
nav_params = pd.read_csv('nav_params.csv')

In [44]:
import SatellitesCoordinateCalculator as SCC
import Transformations
from GNSS import GNSS
import Constants
import numpy as np
from NavTaskUtils import calc_nav_score, calc_eph_score, calc_alm_score

data = {}
# gps_time = gle.receiving_stamp[3].TOW
# gps_time = gle.loc[gle.index[gle.svId == 5], :].receiving_stamp.values[0].TOW
gps_time = int(nav_params.loc[nav_params.index[(nav_params.svId == 5) & (nav_params.gnssId == 'GNSS.GLONASS')], :].receiving_stamp.values[0].split('.')[1].replace('>', ''))
dt = (542812 - 553594)
t1 = (gps_time-dt) % (24*3600)
# for svId in [4, 5, 7, 14, 15, 16, 17, 20, 21, 22]:
# for svId in [5, 14, 15, 22]:
for svId in range(1, 25):
    if svId in [18]:
        continue
    alm = gla[(gla['svId'] == svId) & (gla['gnssId'] == GNSS.GLONASS)]
    alm = alm.loc[alm.index[0], :]
    eph = gle[(gle['svId'] == svId) & (gle['gnssId'] == GNSS.GLONASS)]
    eph = eph.loc[eph.index[0], :]
    nav = nav_params[(nav_params['svId'] == svId) & (nav_params['gnssId'] == 'GNSS.GLONASS')]
    nav = nav.loc[nav.index[0], :]
    nav['prValid'] = bool(nav['prValid'])
    nav['almAvail'] = bool(nav['almAvail'])
    nav['ephAvail'] = bool(nav['ephAvail'])
    if nav.cno == 0 or any(pd.isna(eph)) or any(pd.isna(nav)) or calc_nav_score(nav) < 3 or nav.mpathIndic==3:
        continue
    eph_xyz = SCC.calc_glo_eph_simple(eph, t1, 0)
    alm_xyz = SCC.calc_glo_alm(alm, 8, 209, t1, 209)
    real_rho = np.linalg.norm(np.array(eph_xyz[:3]) - np.array(Constants.ECEF))
    data[svId] = {
        'alm': alm,
        'eph': eph,
        'nav': nav,
        'pr': nav.prMes,
        'pr1': nav.prMes - alm.tau_n * Constants.c,
        'dt': alm.tau_n,
        'real_rho': real_rho,
        'Dt': (real_rho-nav.prMes) / Constants.c,
        'sumDt': (real_rho-nav.prMes) / Constants.c + alm.tau_n,
        'eph_xyz': eph_xyz,
        'eph_lla': Transformations.ecef2lla(*eph_xyz[:3]),
        'alm_xyz': alm_xyz,
        'alm_lla': Transformations.ecef2lla(*alm_xyz[:3]),
        'nav_score': calc_nav_score(nav),
        'eph_score': calc_eph_score(nav),
        'alm_score': calc_alm_score(nav),
    }
print(data)


{1: {'alm': svId                              1
gnssId                 GNSS.GLONASS
receiving_stamp    <TS: 2325.55981>
sfN0                          446.0
sfN1                          446.0
n                               1.0
lambda_n                   2.322228
delta_i_n                  0.028675
eps_n                      0.000359
M_n                             1.0
tau_n                     -0.000092
Cn                              1.0
Hn                              1.0
t_lambda_n              14720.53125
delta_T_n              -2655.259766
delta_T_dot_n             -0.001465
omega_n                    0.265666
ln                              0.0
Name: 0, dtype: object, 'eph': svId                              1
gnssId                 GNSS.GLONASS
receiving_stamp    <TS: 2325.55885>
sfN1                          445.0
sfN2                          445.0
sfN3                          445.0
sfN4                          445.0
tk                         108570.0
tb                   

In [52]:
sats = []
import numpy as np
import Constants
min_sumDt = min(data.values(), key=lambda sat: sat['sumDt'])['sumDt']
print('min_sumDt', min_sumDt)
for svId, sat in data.items():
    x, y, z, *_ = sat['eph_xyz']
    pr = sat['pr1']
    dt = sat['dt']
    # pr = pr - dt * Constants.c
    print(svId, f'{pr:02f}', f'{sat['real_rho']:02f}', f'{sat['sumDt']*1e3:07f}', 'ms, ', f'dDt: {(sat['sumDt']-min_sumDt)*1e6:06f}mks', f'{sat['nav_score'].round(1), sat['eph_score'], sat['alm_score']} scores')
    sats.append((x, y, z, pr))
import Minimizing
import Constants

min_sumDt 0.00206723433660433
1 20522908.475279 21142757.243884 2.067593 ms,  dDt: 0.358600mks (30.0, 10, 10) scores
2 18587332.763750 19207074.026782 2.067234 ms,  dDt: 0.000000mks (7.6, 10, 10) scores
11 20100846.124710 20720848.432596 2.068105 ms,  dDt: 0.870752mks (986.7, 10, 10) scores
12 19392478.197528 20012330.341662 2.070604 ms,  dDt: 3.369860mks (257.8, 10, 10) scores


In [53]:

print("LM")
res = Minimizing.solve_navigation_task_LevMar(sats)
print(res)
print(f'dt: {res.x[-1]/Constants.c}')
print(f'error: {np.linalg.norm(np.array(res.x[:-1]) - np.array(Constants.ECEF))}')

# print("SQP")
# res = Minimizing.solve_navigation_task_SLSQP(sats)
# print(res)
# print(f'error: {np.linalg.norm(np.array(res.x[:-1]) - np.array(Constants.ECEF))}')
# 
# print("DB")
# res = Minimizing.solve_navigation_task_DogBox(sats)
# print(res)
# print(f'error: {np.linalg.norm(np.array(res.x[:-1]) - np.array(Constants.ECEF))}')

LM
LM (55.57103556887867, 38.7780246025531, -3323.2006918189863)
     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [-3.725e-09 -3.725e-09 -3.725e-09  0.000e+00]
           x: [ 2.816e+06  2.263e+06  5.235e+06 -6.232e+05]
        cost: 2.0816681711721685e-17
         jac: [[-4.719e-02 -9.526e-01 -3.007e-01  1.000e+00]
               [-3.456e-01 -3.113e-01 -8.853e-01  1.000e+00]
               [ 3.455e-01 -3.873e-01 -8.548e-01  1.000e+00]
               [-5.274e-01 -7.528e-01 -3.939e-01  1.000e+00]]
        grad: [ 1.761e-10  6.151e-09  7.602e-09 -1.118e-08]
  optimality: 1.1175870895385742e-08
 active_mask: [0 0 0 0]
        nfev: 29
        njev: 28
dt: -0.0020787991302659986
error: 3861.4915780441297


In [21]:

h = gps_time % (24*3600)
print(h/3600)
print(h // 3600)
m = h % 3600
print(m/60)

13.586388888888889
13
35.18333333333333


In [4]:
import numpy as np
from math import pi, floor, sin, cos, atan, sqrt, tan, atan2
import Constants
import Transformations


def get_glo_year(N4, N):
    if 1 <= N <= 366:
        J=1
    elif 367 <= N <= 731:
        J = 2
    elif 732 <= N <= 1096:
        J = 3
    elif 1097<= N <= 1461:
        J = 4
    else:
        J = 0 # TODO - error
    return 1996 + 4 * (N4-1) + (J-1)

def get_glo_dNA(N4, NA, N):
    k = 1461 if N4 != 27 else 1460
    return N - NA - round((N-NA)/k)*k

#TODO: переименовать N в NA в преобраовании ГЛОНАСС 
# t1 = 81300 # 22:35
t1 = 24 * 3600 + 0 * 60
print(f'{t1//3600}h : {(t1%3600//60)}m = ({t1} sec)')
N = 204 # 22.07.2024
# t1, N - время и день, на которые проводится расчет

def calc_glo_alm(alm, eph, t1, N, hard=True):
    GM = Constants.mu
    ae = Constants.ae_glonass
    J20 = Constants.J20_glonass
    OmegaEath = Constants.OmegaEarthDot
    
    NA = eph.NA
    N4 = eph.N4
    t_lambda = alm.t_lambda_n
    dT = alm.delta_T_n
    di = alm.delta_i_n
    dT_dot = alm.delta_T_dot_n
    e = alm.eps_n
    w_A = alm.omega_n
    lambda_A = alm.lambda_n
    
    # для проверки
    # N = 1453
    # t1 = 51300
    # NA = 1452
    # t_lambda = 33571.625
    # dT = 0.01953124999975
    # dT_dot= 6.103515625E-05
    # lambda_A = -0.293967247009277 * pi
    # w_A = 0.57867431640625 * pi
    # e = 0.000432968139648438
    # di = -0.00012947082519531 * pi
    
    # 1 - интервал прогноза dt_pr в сек
    dNa = get_glo_dNA(N4=N4, NA=NA, N=N)
    dt_pr = dNa * 86400 + (t1 - t_lambda)
    # print(1, dt_pr)
    
    # 2 - количество целых витков W на интервале прогноза
    Tdr_avg = 43200
    W = floor(dt_pr / (Tdr_avg + dT))
    # print(2, W)
    
    # 3 - текущее наклонение
    i_avg = 63 / 180 * pi
    i = di + i_avg
    # print(3, i)
    #TODO: результат не сходится с примеров в документации
    # #i = 1.1305666106990377
    
    # 4 - средний драконический период на витке W+1 и среднее движение
    Tdr = Tdr_avg + dT + (2*W + 1) * dT_dot
    n = 2*pi / Tdr
    # print(4, Tdr, n)
    
    # 5 - рассчитывается большая полуось последовательными приближениями (m=0, 1, 2...)
    T1 = Tdr
    a0 = -1
    a = 0
    p = 0
    while abs(a0-a) > 1e-3:
        a0 = a
        a = ((T1/2/pi)**2 * GM) ** (1.0/3.0)
        p = a * (1-e**2)
        Ttt = (2-2.5*sin(i)**2) * ((1-e**2) ** 1.5 / (1 + e * cos(w_A * pi)) ** 2) + (1 + e * cos(w_A * pi)) ** 3 / (1 - e ** 2)
        Tdown = 1 - 1.5 * J20 * (ae/p)** 2 * Ttt
        T1 = Tdr / Tdown
    # print(5, a1, p1, T1)
    
    # 6 - текущее значение долготы восходящего узла и аргумента перигея
    lmb = lambda_A - (OmegaEath + 1.5*J20*n*(ae/p)**2*cos(i))* dt_pr
    w = w_A - 0.75 * J20 * n * (ae/p)**2 * (1-5*cos(i)**2) * dt_pr
    # print(6, lmb, w)
    
    # 7 - среднее значение долготы на момент прохождения восходящего узла
    E0 = -2 * atan(sqrt((1-e)/(1+e)) * tan(w/2))
    L1 = w + E0 - e*sin(E0)
    # print(7, E0, L1)
    
    # 8 - текущее значение долготы НКА
    L = L1 + n * (dt_pr - (Tdr_avg + dT)*W - dT_dot*W**2)
    L2 = L
    # print(8, L)
    
    # 9 - для повышения точности, можно опустить
    def calc_derivatives(Lk):
        B = 1.5 * J20 * (ae/a)**2
        da_a_k = 2*B*(1-1.5*sin(i)**2) * (l*cos(Lk) + h * sin(Lk)) + B * sin(i)**2 * (0.5*h*sin(Lk) - 0.5*l*cos(Lk) + cos(2*Lk) + 3.5*l*cos(3*Lk) + 3.5*h*sin(3*Lk))
        da_k = da_a_k * a
        dh_k = B * (1-1.5*sin(i)**2) * (sin(Lk)+1.5*l*sin(2*Lk)-1.5*h*cos(2*Lk)) - 0.25*B*sin(i)**2 * (sin(Lk)-(7.0/3.0) * sin(3*Lk) + 5*l*sin(2*Lk) - 8.5*l*sin(4*Lk) + 8.5*h*cos(4*Lk) + h*cos(2*Lk)) + (-0.5*B*cos(i)**2*l*sin(2*Lk))
        dl_k = B * (1-1.5*sin(i)**2) * (cos(Lk)+1.5*l*cos(2*Lk)+1.5*h*sin(2*Lk)) - 0.25*B*sin(i)**2 * (-cos(Lk)-(7.0/3.0)* cos(3*Lk) - 5*h*sin(2*Lk) - 8.5*l*cos(4*Lk) - 8.5*h*sin(4*Lk) + l*cos(2*Lk)) + 0.5*B*cos(i)**2*h*sin(2*Lk)
        dlmb_k = -B*cos(i)*(3.5*l*sin(Lk)-2.5*h*cos(Lk)-0.5*sin(2*Lk)-(7.0/6.0)*l*sin(3*Lk)+(7.0/6.0)*h*cos(3*Lk))
        di_k = 0.5*B*sin(i)*cos(i)*(-l*cos(Lk)+h*sin(Lk) +cos(2*Lk) + (7.0/3.0)*l*cos(3*Lk)+(7.0/3.0)*h*sin(3*Lk))
        dL_k = 2*B*(1-1.5*sin(i)**2) * (1.75*l*sin(Lk) - 1.75*h*cos(Lk)) + 3*B*sin(i)**2 * (-(7.0/24.0)*h*cos(Lk)-(7.0/24.0)*l*sin(Lk) -(49.0/72.0)*h*cos(3*Lk) + (49.0/72.0)*l*sin(3*Lk) + 0.25*sin(2*Lk)) + B*cos(i)**2 * (3.5*l*sin(Lk)-2.5*h*cos(Lk)-0.5*sin(2*Lk)-(7.0/6.0)*l*sin(3*Lk) + (7.0/6.0)*h*cos(3*Lk))
        return da_k, dh_k, dl_k, dlmb_k, di_k, dL_k
    if hard:
        h = e * sin(w)
        l = e * cos(w)
        
        da1, dh1, dl1, dlmb1, di1, dL1 = calc_derivatives(Lk=L1)
        da2, dh2, dl2, dlmb2, di2, dL2 = calc_derivatives(Lk=L2)
        
        a = a + da2 - da1
        h = h + dh2 - dh1
        l = l + dl2 - dl1
        e = sqrt(h**2+l**2)
        i = i + di2 - di1
        lmb = lmb + dlmb2 - dlmb1
        w = atan2(h, l) #atan(h/l)
        L = L + dL2 - dL1
    
    # 10 - эксцентрическая аномалия
    E1 = 10 + E0
    while abs(E1-E0) > 1e-9:
        E0 = E1
        E1 = L - w + e * sin(E0)
    E = E1
    
    # 11 истинняа аномалия и аргумент широты
    nu = 2 * atan( sqrt((1+e)/(1-e)) * tan(E/2))
    u = nu + w
    
    # 12 - координаты ЦМ в прямоугольной ск
    p = a * (1-e**2)
    r = p / (1+e * cos(nu))
    X = r * (cos(lmb)*cos(u)-sin(lmb)*sin(u)*cos(i))
    Y = r * (sin(lmb)*cos(u)+cos(lmb)*sin(u)*cos(i))
    Z = r * sin(u) * sin(i)
    
    # Vr = sqrt(GM/p) * e * sin(nu)
    # Vu = sqrt(GM/p) * (1+e*cos(nu))
    # Vx = Vr * (cos(lmb)*cos(u)-sin(lmb)*sin(u)*cos(i)) - Vu * (cos(lmb)*sin(u)+sin(lmb)*cos(u)*cos(i)) + OmegaEath * Y
    # Vy = Vr * (sin(lmb)*cos(u)+cos(lmb)*sin(u)*cos(i)) - Vu * (sin(lmb)*sin(u)-cos(lmb)*cos(u)*cos(i)) - OmegaEath * X
    # Vz = Vr * (sin(u)*sin(i)) + Vu * (cos(u)*sin(i))
    
    return X, Y, Z

prMes = {
    7: 17931698.162640072,
    16: 17247315.440587413,
    17: 19352936.817768987,
    24: 21353624.2500098,
}
# import Transformations
# data = []
# eph = gle.loc[6]
# for svId in prMes.keys():
#     alm = gla.loc[svId-1]
#     # eph = gle.loc[svId-1]
#     assert svId == alm.svId# and svId == eph.svId
#     x, y, z = calc_glo_alm(alm, eph, t1, N)
#     print(svId, tuple(np.array(Transformations.ecef2lla(x, y, z)).round(2)), round(prMes[svId], -3))
#     data.append((x, y, z, prMes[svId]))

24h : 0m = (86400 sec)


In [5]:
sv = 4
eph = gle3.loc[sv-1]
eph2 = gle5.loc[sv-1]
# eph3 = gle4.loc[sv-1]
alm = gla3.loc[sv-1]
alm2 = gla5.loc[sv-1]
N = 205
t1 = eph2.tb
# xa, ya, za = calc_glo_alm(alm, eph, t1, N, hard=True)
xa, ya, za = calc_glo_alm(gla4.loc[sv-1], eph, t1, N, hard=True)
# xa2, ya2, za2 = calc_glo_alm(alm2, eph2, t1, N, hard=True)
xe, ye, ze, vx, vy, vz = calc_glo_eph_simple(eph, t1, N).y[:, -1]
x2, y2, z2, vx2, vy2, vz2 = eph2[['x', 'y', 'z', 'dx', 'dy', 'dz']] * 1000

print(xa, ya, za, Transformations.ecef2lla(xa, ya, za), 'alm calced')
# print(xa2, ya2, za2, Transformations.ecef2lla(xa2, ya2, za2))
print(xe, ye, ze, vx, vy, vz, 'eph calced')
print(x2, y2, z2, vx2, vy2, vz2, 'from eph')
# print(eph3[['x', 'y', 'z', 'dx', 'dy', 'dz']])

NameError: name 'gle3' is not defined

In [6]:
import numpy as np
from math import pi, floor, sin, cos, atan, sqrt, tan, atan2
import Constants
import Transformations
from scipy.integrate import RK45 , solve_ivp, odeint

def calc_glo_eph_simple(eph, t1, N):
    X0 = eph.x
    Vx0 = eph.dx
    ddx = eph.ddx
    Y0 = eph.y
    Vy0 = eph.dy
    ddy = eph.ddy
    Z0 = eph.z
    Vz0 = eph.dz
    ddz = eph.ddz
    tb = eph.tb
    
    maxT = lambda ti: ti - round((ti-tb)/86400)*86400
    
    ae = Constants.ae_glonass
    GM = Constants.mu
    J20 = Constants.J20_glonass
    OmegaEarth = Constants.OmegaEarthDot
    
    y0 = np.array([X0, Y0, Z0, Vx0, Vy0, Vz0]) * 1000
    def func(t, y):
        X, Y, Z, Vx, Vy, Vz = y
        r = sqrt(X**2+Y**2+Z**2)
        dVx = -GM/r**3 * X - 1.5*J20*GM*ae**2/r**5 * X * (1 - 5*Z**2/r**2) + OmegaEarth**2 * X + 2 * OmegaEarth * Vy + ddx
        dVy = -GM/r**3 * Y - 1.5*J20*GM*ae**2/r**5 * Y * (1 - 5*Z**2/r**2) + OmegaEarth**2 * Y - 2 * OmegaEarth * Vx + ddy
        dVz = -GM/r**3 * Z - 1.5*J20*GM*ae**2/r**5 * Z * (3 - 5*Z**2/r**2) + ddz
        # print(t, np.array([X, Y, Z, Vx, Vy, Vz, dVx, dVy, dVz]).round(0))
        return np.array([Vx, Vy, Vz, dVx, dVy, dVz])
    runer = RK45(func, tb, y0, t1, first_step=1, max_step=1)
    return runer
    # while runer.status == 'running':
    #     runer.step()
    #     # print(runer.t, runer.y.round(3))
    # # # print(runer.t, runer.y.round(3))
    # # return runer.y
    # # res = solve_ivp(func, (tb, t1), y0, method='RK45', max_step=10, first_step=1)
    # return runer

sv = 17
eph = gle2.loc[sv-1]
# t1 = 81300 # 22:35
t1 = 23 * 3600 + 43 * 60
print(f'{t1//3600}h : {(t1%3600//60)}min')
N = 204 # 22.07.2024
# t1, N - время и день, на которые проводится расчет
runer = calc_glo_eph_simple(eph, t1, N)
# res = calc_glo_eph_simple(eph, t1, N)
# x, y, z, Vx, Vy, Vz = res.y#[:, -1]
# print(eph.svId)
# print(x, y, z)
# print(Transformations.ecef2lla(x, y, z))
print(runer.t)
print(t1)
while runer.t < t1:
    runer.step()
    while runer.t%60 != 0:
        runer.step()
        # print(runer.t, end='\t')
    # print()
    print(runer.t, runer.y)
    


23h : 43min
81900.0
85380
81960.0 [-1.09534512e+07  2.49565522e+06  2.29099202e+07  4.80056502e+02
 -3.10115456e+03  5.62967745e+02]
82020.0 [-1.09250948e+07  2.30937945e+06  2.29427085e+07  4.65143078e+02
 -3.10798133e+03  5.29965433e+02]
82080.0 [-1.08976358e+07  2.12270426e+06  2.29735151e+07  4.50140558e+02
 -3.11446841e+03  4.96916469e+02]
82140.0 [-1.08710796e+07  1.93565006e+06  2.30023376e+07  4.35052928e+02
 -3.12061459e+03  4.63823701e+02]
82200.0 [-1.08454311e+07  1.74823735e+06  2.30291732e+07  4.19884191e+02
 -3.12641871e+03  4.30689980e+02]
82260.0 [-1.08206951e+07  1.56048668e+06  2.30540196e+07  4.04638364e+02
 -3.13187968e+03  3.97518162e+02]
82320.0 [-1.07968760e+07  1.37241867e+06  2.30768746e+07  3.89319480e+02
 -3.13699646e+03  3.64311106e+02]
82380.0 [-1.07739781e+07  1.18405401e+06  2.30977363e+07  3.73931584e+02
 -3.14176807e+03  3.31071673e+02]
82440.0 [-1.07520055e+07  9.95413425e+05  2.31166027e+07  3.58478737e+02
 -3.14619358e+03  2.97802730e+02]
82500.0 [-1

In [17]:
runer.t_bound += 60
runer.status='running'
print(runer.t, runer.y)
runer.step()
print(runer.t, runer.y)

85380.0 [-1.08255562e+07 -8.14888721e+06  2.16048830e+07 -3.92504598e+02
 -2.93599453e+03 -1.30872187e+03]
85381.0 [-1.08259488e+07 -8.15182310e+06  2.16035740e+07 -3.92730240e+02
 -2.93578478e+03 -1.30924104e+03]


In [7]:
def calc_julian_day(N4: int, N: int) -> tuple[int, int, int, int, float]:
    JD0 = 1461 * (N4-1) + N + 2450082.5
    JDN = JD0 + 0.5
    a = JDN + 32044
    b = (4*a+3)//146097
    c = a - (146097*b)//4
    d = (4*c+3) // 1461
    e =c - (1461*d)//4
    m = (5*e+2)//153
    
    day: int = e - (153*m+2) // 5 + 1
    month: int = m + 3 - 12 * (m//10)
    year: int = 100*b + d - 4800 + (m//10)
    day_of_week: int = JDN % 7 + 1
    
    ERA = 2 * pi * (0.7790572732640 + 1.00273781191135448 * (JD0 - 2451545.0))
    dT = (JD0 - 2451545.0) / 36525
    GMST = ERA + 0.0000000703270726 + 0.0223603658710194 * dT + 0.0000067465784654 * dT**2  - 0.0000000000021332 * dT**3  -0.0000000001452308 *dT**4 - 0.0000000000001784 *dT**5 
    return day, month, year, day_of_week, GMST
N = 251
N4 = 5
calc_julian_day(N4, N)

(7.0, 9.0, 2012.0, 5.0, 29191.4428297696)

In [8]:
import Minimizing
res1 = Minimizing.solve_navigation_task_SLSQP(data)
print(res1)
dt = res1.x[-1] / Constants.c
print(f'dt: {dt}')
import Transformations
lla = Transformations.ecef2lla(*res1.x[:-1])
print(lla)

NameError: name 'data' is not defined

In [9]:
res2 = Minimizing.solve_navigation_task_LevMar(data)

print(res2)
dt = res2.x[-1] / Constants.c

print(f'dt: {dt}')
lla = Transformations.ecef2lla(*res2.x[:-1])
print(lla)
print(sum(res2.fun))

NameError: name 'data' is not defined

In [10]:
res3 = Minimizing.solve_navigation_task_GO(data)
print(res3)
dt = res3.x[-1] / Constants.c

print(f'dt: {dt}')
lla = Transformations.ecef2lla(*res3.x[:-1])
print(lla)

NameError: name 'data' is not defined

In [13]:
import numpy as np

class RK45Solver:
    order = 5
    error_estimator_order = 4
    n_stages = 6
    C = np.array([0, 1/5, 3/10, 4/5, 8/9, 1])
    A = np.array([
        [0, 0, 0, 0, 0],
        [1/5, 0, 0, 0, 0],
        [3/40, 9/40, 0, 0, 0],
        [44/45, -56/15, 32/9, 0, 0],
        [19372/6561, -25360/2187, 64448/6561, -212/729, 0],
        [9017/3168, -355/33, 46732/5247, 49/176, -5103/18656]
    ])
    B = np.array([35/384, 0, 500/1113, 125/192, -2187/6784, 11/84])
    E = np.array([-71/57600, 0, 71/16695, -71/1920, 17253/339200, -22/525,
                  1/40])
    P = np.array([
        [1, -8048581381/2820520608, 8663915743/2820520608,
         -12715105075/11282082432],
        [0, 0, 0, 0],
        [0, 131558114200/32700410799, -68118460800/10900136933,
         87487479700/32700410799],
        [0, -1754552775/470086768, 14199869525/1410260304,
         -10690763975/1880347072],
        [0, 127303824393/49829197408, -318862633887/49829197408,
         701980252875 / 199316789632],
        [0, -282668133/205662961, 2019193451/616988883, -1453857185/822651844],
        [0, 40617522/29380423, -110615467/29380423, 69997945/29380423]])
    
    
    def __init__(self, fun, y0, t0, step=1):
        self.fun = fun
        self.t = [t0]
        self.y = [y0]
        self.f = fun(t0, y0)
        self.step = step
        self.n = y0.size
        self.K = np.empty((self.n_stages + 1, self.n), dtype=y0.dtype)
    
    def get(self, t):
        if t < self.t[0]:
            return None
        while t > self.t[-1]:
            self.next()
        return self.y[-1]
    
    def next(self):
        self.t.append(self.t[-1] + self.step)
        y, self.f = rk_step(self.fun, self.t[-1], self.y[-1], self.f, self.step, self.A, self.B, self.C, self.K)
        self.y.append(y)
        return self.t[-1], self.y[-1]


def rk_step(fun, t, y, f, h, A, B, C, K):
    K[0] = f
    for s, (a, c) in enumerate(zip(A[1:], C[1:]), start=1):
        dy = np.dot(K[:s].T, a[:s]) * h
        K[s] = fun(t + c * h, y + dy)
    y_new = y + h * np.dot(K[:-1].T, B)
    f_new = fun(t + h, y_new)
    K[-1] = f_new
    return y_new, f_new

def get_glo_eph_simple_func(eph):
    X0 = eph.x
    Vx0 = eph.dx
    ddx = eph.ddx
    Y0 = eph.y
    Vy0 = eph.dy
    ddy = eph.ddy
    Z0 = eph.z
    Vz0 = eph.dz
    ddz = eph.ddz
    tb = eph.tb
    
    maxT = lambda ti: ti - round((ti-tb)/86400)*86400
    
    ae = Constants.ae_glonass
    GM = Constants.mu
    J20 = Constants.J20_glonass
    OmegaEarth = Constants.OmegaEarthDot
    
    y0 = np.array([X0, Y0, Z0, Vx0, Vy0, Vz0]) * 1000
    def func(t, y):
        X, Y, Z, Vx, Vy, Vz = y
        r = sqrt(X**2+Y**2+Z**2)
        dVx = -GM/r**3 * X - 1.5*J20*GM*ae**2/r**5 * X * (1 - 5*Z**2/r**2) + OmegaEarth**2 * X + 2 * OmegaEarth * Vy + ddx
        dVy = -GM/r**3 * Y - 1.5*J20*GM*ae**2/r**5 * Y * (1 - 5*Z**2/r**2) + OmegaEarth**2 * Y - 2 * OmegaEarth * Vx + ddy
        dVz = -GM/r**3 * Z - 1.5*J20*GM*ae**2/r**5 * Z * (3 - 5*Z**2/r**2) + ddz
        # print(t, np.array([X, Y, Z, Vx, Vy, Vz, dVx, dVy, dVz]).round(0))
        return np.array([Vx, Vy, Vz, dVx, dVy, dVz])
    return func, y0, eph.tb
func, y0, tb = get_glo_eph_simple_func(eph)
runer = RK45Solver(func, y0, tb)
t, y = 0, 0
for i in range(100):
    for j in range(60):
        t, y = runer.next()
    print(t, y)

81960.0 [-1.09534512e+07  2.49565522e+06  2.29099202e+07  4.80056502e+02
 -3.10115456e+03  5.62967745e+02]
82020.0 [-1.09250948e+07  2.30937945e+06  2.29427085e+07  4.65143078e+02
 -3.10798133e+03  5.29965433e+02]
82080.0 [-1.08976358e+07  2.12270426e+06  2.29735151e+07  4.50140558e+02
 -3.11446841e+03  4.96916469e+02]
82140.0 [-1.08710796e+07  1.93565006e+06  2.30023376e+07  4.35052928e+02
 -3.12061459e+03  4.63823701e+02]
82200.0 [-1.08454311e+07  1.74823735e+06  2.30291732e+07  4.19884191e+02
 -3.12641871e+03  4.30689980e+02]
82260.0 [-1.08206951e+07  1.56048668e+06  2.30540196e+07  4.04638364e+02
 -3.13187968e+03  3.97518162e+02]
82320.0 [-1.07968760e+07  1.37241867e+06  2.30768746e+07  3.89319480e+02
 -3.13699646e+03  3.64311106e+02]
82380.0 [-1.07739781e+07  1.18405401e+06  2.30977363e+07  3.73931584e+02
 -3.14176807e+03  3.31071673e+02]
82440.0 [-1.07520055e+07  9.95413425e+05  2.31166027e+07  3.58478737e+02
 -3.14619358e+03  2.97802730e+02]
82500.0 [-1.07309619e+07  8.06517716e

In [14]:
from GNSS import GNSS
from enum import Enum

class CalcMode(Enum):
    alm = 0
    eph = 1
    pass

def calc_gps_eph(data, t1, N):
    pass
def calc_gps_alm(data, t1, N):
    pass
# def calc_glo_alm(data, t1, N):
# def calc_glo_alm(alm, t1, N, hard=True):
#     pass
# def get_glo_eph_func(eph):
#     pass
# def 

class SatellitesCoordinateCalculator:
    runer = None
    def __init__(self, data, gnssId: GNSS, mode: CalcMode):
        self.next_func = self.choose_func(data, gnssId, mode)
    
    def choose_func(self, data, gnssId, mode):
        match (gnssId, mode):
            case (GNSS.GPS, CalcMode.alm):
                return lambda t1, N: calc_gps_alm(data, t1, N)
            case (GNSS.GPS, CalcMode.eph):
                return lambda t1, N: calc_gps_eph(data, t1, N)
            case (GNSS.GLONASS, CalcMode.alm):
                return lambda t1, N: calc_glo_alm(data, t1, N, hard=True)
            case (GNSS.GLONASS, CalcMode.eph):
                self.runer = RK45Solver(*get_glo_eph_simple_func(data))
                return lambda t1, N: self.runer.get(t1) 
    def get(self, t1, N=0 ):
        return self.next_func(t1, N)
# TODO: перевести все в метки TimeStamp
scc = SatellitesCoordinateCalculator(eph, GNSS.GLONASS, CalcMode.eph)
scc.get(24 *3600)
        

array([-1.13358194e+07, -1.10194242e+07,  2.00059778e+07, -5.99965540e+02,
       -2.67820120e+03, -1.81989374e+03])

In [15]:
scc.get(24*3600-10)

array([-1.13358194e+07, -1.10194242e+07,  2.00059778e+07, -5.99965540e+02,
       -2.67820120e+03, -1.81989374e+03])